# About

test notebook to explore cosine similarity & tfidf


In [243]:
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', -1)

## sample

In [70]:
t1 = questions[questions["accepted_answer_id"] > 0]
questions_sample = t1.sample(600)

## corpus

In [226]:
# constructs corpus
# with question id, title, accepted answer id, answer body
def construct_corpus(questions, answers = answers):
    t1 = questions[["id", "title", "accepted_answer_id"]].rename(columns = {"id" : "q_id", "title" : "q_title"})
    t2 = answers[["id", "body"]].rename(columns = {"id" : "a_id", "body" : "a_body"})
    t3 = t1.merge(t2, left_on = "accepted_answer_id", right_on = "a_id", how = "inner").drop(columns = "a_id")
    return(t3)

In [244]:
corpus = construct_corpus(questions_sample)
corpus.head(3)

,q_id,q_title,accepted_answer_id,a_body
0,40482556,is it possible to use web view as a html reader (offline)?,40482678.0,"<p>You could just put the file into your assets folder in the source code, and then do this:</p>\r\n\r\n<pre><code>WebView webView = (WebView)findViewById(R.id.webView1);\r\nwebview.loadUrl(""file:///android_assets/file.html"");\r\n</code></pre>\r\n\r\n<p><strong>For display PDF:</strong></p>\r\n\r\n<p>Since API Level 21 (Lollipop) Android provides a PdfRenderer class:</p>\r\n\r\n<pre><code>// create a new renderer\r\nPdfRenderer renderer = new PdfRenderer(getSeekableFileDescriptor());\r\n\r\n// let us just render all pages\r\nfinal int pageCount = renderer.getPageCount();\r\nfor (int i = 0; i &lt; pageCount; i++) {\r\n Page page = renderer.openPage(i);\r\n\r\n // say we render for showing on the screen\r\n page.render(mBitmap, null, null, Page.RENDER_MODE_FOR_DISPLAY);\r\n\r\n // do stuff with the bitmap\r\n\r\n // close the page\r\n page.close();\r\n}\r\n\r\n// close the renderer\r\nrenderer.close();\r\n</code></pre>\r\n\r\n<p>This sample demonstrates <a href=""https://github.com/googlesamples/android-PdfRendererBasic"" rel=""nofollow noreferrer"">how to display PDF document on screen using the PdfRenderer introduced in Android 5.0 Lollipop.</a></p>\r\n\r\n<p>For older APIs, checkout <a href=""https://github.com/barteksc/AndroidPdfViewer"" rel=""nofollow noreferrer"">AndroidPdfViewer library</a>, works on API 11 and higher:</p>\r\n\r\n<pre><code>pdfView.fromUri(Uri)\r\nor\r\npdfView.fromFile(File)\r\nor\r\npdfView.fromAsset(String)\r\n .pages(0, 2, 1, 3, 3, 3) // all pages are displayed by default\r\n .enableSwipe(true)\r\n .swipeHorizontal(false)\r\n .enableDoubletap(true)\r\n .defaultPage(0)\r\n .onDraw(onDrawListener)\r\n .onLoad(onLoadCompleteListener)\r\n .onPageChange(onPageChangeListener)\r\n .onPageScroll(onPageScrollListener)\r\n .onError(onErrorListener)\r\n .enableAnnotationRendering(false)\r\n .password(null)\r\n .scrollHandle(null)\r\n .load();\r\n</code></pre>"
1,10066328,C++: Comparing strings lexicographical,10066372.0,"<p>Yes.</p>\r\n\r\n<p>String's comparison operators are defined in terms of its <code>traits::compare</code> (that is <code>char_traits&lt;char&gt;::compare</code>) (C++03 21.3.6.8) which is specified to return a value based on the lexicographical ordering of its arguments (21.1.1).</p>\r\n\r\n<blockquote>\r\n <p>X::compare(p,q,n) ... yields: 0 if for each i in [0,n),\r\n X::eq(p[i],q[i]) is true; else, a negative\r\n value if, for some j in [0,n),\r\n X::lt(p[j],q[j]) is true and for each i in\r\n [0,j) X::eq(p[i],q[i]) is true; else a positive value.</p>\r\n</blockquote>\r\n\r\n<p>In effect, it means comparing string must not be locale sensitive (which could be non-lexicographical in some locales, such as mine).</p>"
2,50038413,Fill bubbles with color in svyplot() in R (survey package),50038857.0,"<p>We can do the following:</p>\r\n\r\n<pre><code>getcol &lt;- function(df) c(""goldenrod"",""tomato"",""sienna"")[as.numeric(df$stype)]\r\n\r\nsvyplot(api00 ~ api99,\r\n design = dstrat,\r\n basecol = getcol,\r\n style = ""bubble"",\r\n alpha = c(0,1),\r\n pch = 21,\r\n bg = getcol(dstrat$variables))\r\n</code></pre>\r\n\r\n<p><a href=""https://i.stack.imgur.com/oWOxu.png"" rel=""nofollow noreferrer""><img src=""https://i.stack.imgur.com/oWOxu.png"" alt=""enter image description here""></a></p>\r\n\r\n<p>Some comments:</p>\r\n\r\n<ol>\r\n<li>In base R's <code>plot</code>, when using <code>pch = 21</code> you can set the outline colour with <code>col</code> and fill colour with <code>bg</code>.</li>\r\n<li>Here, <code>basecol</code> is an <code>svyplot</code>-specific argument, which internally sets the outline colour <code>col</code>.</li>\r\n<li>We can use the same function that we used for <code>basecol</code> to fill circles with the matching colour, by recognising that <code>dstrat$variables</code> is the <code>data.frame</code> that contains column <code>stype</code>.</li>

# Cosine Similarity for Existing Questions

In [230]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer(analyzer='word', ngram_range=(1,3), min_df = 0, stop_words = 'english')
tfidf_matrix = tf.fit_transform(corpus.q_title)

In [240]:
from sklearn.metrics.pairwise import linear_kernel

def find_similar(tfidf_matrix, index, top_n = 3):
    cosine_similarities = linear_kernel(tfidf_matrix[index:index+1], tfidf_matrix).flatten()
    related_docs_indices = [i for i in cosine_similarities.argsort()[::-1] if i != index]
    return [(index, cosine_similarities[index]) for index in related_docs_indices][0:top_n]

In [309]:
def decorate_answers(similar_qs, corpus = corpus):
    t1 = pd.DataFrame(similar_qs, columns = ["q_index", "cos_similarity"]).set_index(["q_index"])
    t2 = corpus.iloc[t1.index]
    t3 = pd.concat([t1, t2], axis = 1)
    return(t3)

In [256]:
q_index = 101
q1 = corpus.iloc[q_index]
print(q1.q_title)
similar_qs = find_similar(tfidf_matrix, q_index, top_n = 3)
print(similar_qs)

How to point to the center coordinates of a specific object in 2D in Unity?
[(342, 0.10026783233746424), (340, 0.08168268913355363), (372, 0.07763675441482287)]


In [310]:
decorate_answers(similar_qs)

,cos_similarity,q_id,q_title,accepted_answer_id,a_body
342,0.100268,21854110,how to rotate an imageview in android on center point,21854206.0,"<pre><code>public class RotateImageView extends ImageView {\r\n\r\nprivate Animation mRotation;\r\n public bool isAnimating = false;\r\n\r\npublic RotateImageView(Context context) {\r\n super(context);\r\n Init(null);\r\n}\r\n\r\npublic RotateImageView(Context context, AttributeSet attrs) {\r\n super(context, attrs);\r\n Init(attrs);\r\n}\r\n\r\n@TargetApi(Build.VERSION_CODES.HONEYCOMB)\r\npublic RotateImageView(Context context, AttributeSet attrs, int defStyle) {\r\n super(context, attrs, defStyle);\r\n Init(attrs);\r\n}\r\n\r\nprivate void Init(AttributeSet attrs) {\r\n startAnimation();\r\n}\r\n\r\npublic void startAnimation() {\r\n if (mRotation == null) {\r\n mRotation = AnimationUtils.loadAnimation(getContext(), R.anim.rotate);\r\n mRotation.setRepeatCount(Animation.INFINITE);\r\n }\r\n this.startAnimation(mRotation);\r\n isAnimating = true;\r\n}\r\n\r\npublic void stopAnimation() {\r\n if (mRotation != null)\r\n clearAnimation();\r\n isAnimating = false;\r\n}\r\n\r\n@Override\r\npublic void setVisibility(int visibility) {\r\n if (visibility == GONE || visibility == INVISIBLE) {\r\n clearAnimation();\r\n } else if (visibility == VISIBLE) {\r\n startAnimation(mRotation);\r\n }\r\n super.setVisibility(visibility);\r\n}\r\n</code></pre>\r\n\r\n<p>}</p>\r\n\r\n<p>rotate.xml</p>\r\n\r\n<pre><code> &lt;?xml version=""1.0"" encoding=""utf-8""?&gt;\r\n&lt;rotate xmlns:android=""http://schemas.android.com/apk/res/android""\r\n android:duration=""5000""\r\n android:fromDegrees=""0""\r\n android:interpolator=""@android:anim/linear_interpolator""\r\n android:pivotX=""50%""\r\n android:pivotY=""50%""\r\n android:startOffset=""0""\r\n android:toDegrees=""360"" /&gt;\r\n</code></pre>\r\n\r\n<p><strong>EDITED</strong></p>\r\n\r\n<pre><code> RotateImageView image = new RotateImageView(Context);\r\n image.addOnClickListener(new View.OnClickListener(){\r\n if(image.isAnimating)\r\n image.stopAnimating();\r\n else \r\n image.startAnimating();\r\n}\r\n</code></pre>"
340,0.081683,45123186,Find out an object's box actual 4 edges scaled to the screen/camera size (Unity 2D),45124972.0,"<p>As Draco18s pointed out, you convert <code>pos</code> to screenspace but not <code>rect</code>.</p>\r\n\r\n<p>If you look at unity's API docs for <code>WorldToScreenPoint()</code>\r\nit says that it </p>\r\n\r\n<blockquote>\r\n <p>Transforms position from world space into screen space. Screenspace is\r\n defined in pixels.</p>\r\n</blockquote>\r\n\r\n<p>whereas <code>transform.position</code> returns</p>\r\n\r\n<blockquote>\r\n <p>The position of the transform in world space.</p>\r\n</blockquote>\r\n\r\n<p>What you have is a problem of combining screenspace, measured in pixels, and world space, which is totally different.</p>\r\n\r\n<p>Because you are using <code>pixelRect</code>, I would recommend changing <code>var rect = image.transform.rect;</code>\r\nto be <code>var rect = Camera.main.WorldToScreenPoint(image.transform.position);</code></p>\r\n\r\n<p>You will then of course have to also check the other corners in a similar way, and if all 4 corners return true then the the object is inside the other.</p>"
372,0.077637,50272399,Three.js - 2D object in 3D space (by Vertices),50274103.0,"<p>Just a concept, using a quaternion:</p>\r\n\r\n<p><div class=""snippet"" data-lang=""js"" data-hide=""false"" data-console=""false"" data-babel=""false"">\r\n<div class=""snippet-code"">\r\n<pre class=""snippet-code-js lang-js prettyprint-override""><code>var scene = new THREE.Scene();\r\nvar camera = new THREE.PerspectiveCamera(60, window.innerWidth / window.innerHeight, 1, 1000);\r\ncamera.position.set(2, 2, 4).setLength(40);\r\nvar renderer = new THREE.WebGLRenderer({\r\n antialias: true\r\n});\r\nrenderer.setSize(window.innerWidth, window.innerHeight);\r\nrenderer.setClearColor(0x101010);\r\ndocument.body.appendChild(renderer.do

# Cosine Similarity for User-Input Questions

In [312]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer(analyzer='word', ngram_range=(1,3), min_df = 0, stop_words = 'english')
tfidf_matrix = tf.fit_transform(corpus.q_title)

In [321]:
from sklearn.metrics.pairwise import linear_kernel

def find_similar(tfidf_matrix, question_input, top_n = 3):
    t1 = tf.transform([question_input])
    cosine_similarities = linear_kernel(t1, tfidf_matrix).flatten()
    related_docs_indices = [i for i in cosine_similarities.argsort()[::-1]]
    return [(index, cosine_similarities[index]) for index in related_docs_indices][0:top_n]

In [322]:
def decorate_answers(similar_qs, corpus = corpus):
    t1 = pd.DataFrame(similar_qs, columns = ["q_index", "cos_similarity"]).set_index(["q_index"])
    t2 = corpus.iloc[t1.index]
    t3 = pd.concat([t1, t2], axis = 1)
    return(t3)

In [323]:
similar_qs = find_similar(tfidf_matrix, "How to point to the center coordinates of a specific object in 2D in Unity?")
similar_qs

[(101, 1.0000000000000002),
 (342, 0.10026783233746427),
 (340, 0.08168268913355364)]

In [317]:
decorate_answers(similar_qs)

,cos_similarity,q_id,q_title,accepted_answer_id,a_body
101,1.000000,46894448,How to point to the center coordinates of a specific object in 2D in Unity?,46894801.0,"<p>You have the possibility to assign the ""forward"" vector of your 2D element using <code>transform.right = ....</code></p>\r\n\r\n<pre><code>Transform target = GameObject.Find(""Center_Planet"").transform ;\r\n\r\n// ...\r\n\r\nfor (int i = 0; i &lt; missile_num; ++i)\r\n{\r\n // ... \r\n missile_prefab_instantiate.transform.right = target.position - missile_prefab_instantiate.transform.position ;\r\n}\r\nyield return new WaitForSeconds(0.9f);\r\n\r\n// ...\r\n</code></pre>"
342,0.100268,21854110,how to rotate an imageview in android on center point,21854206.0,"<pre><code>public class RotateImageView extends ImageView {\r\n\r\nprivate Animation mRotation;\r\n public bool isAnimating = false;\r\n\r\npublic RotateImageView(Context context) {\r\n super(context);\r\n Init(null);\r\n}\r\n\r\npublic RotateImageView(Context context, AttributeSet attrs) {\r\n super(context, attrs);\r\n Init(attrs);\r\n}\r\n\r\n@TargetApi(Build.VERSION_CODES.HONEYCOMB)\r\npublic RotateImageView(Context context, AttributeSet attrs, int defStyle) {\r\n super(context, attrs, defStyle);\r\n Init(attrs);\r\n}\r\n\r\nprivate void Init(AttributeSet attrs) {\r\n startAnimation();\r\n}\r\n\r\npublic void startAnimation() {\r\n if (mRotation == null) {\r\n mRotation = AnimationUtils.loadAnimation(getContext(), R.anim.rotate);\r\n mRotation.setRepeatCount(Animation.INFINITE);\r\n }\r\n this.startAnimation(mRotation);\r\n isAnimating = true;\r\n}\r\n\r\npublic void stopAnimation() {\r\n if (mRotation != null)\r\n clearAnimation();\r\n isAnimating = false;\r\n}\r\n\r\n@Override\r\npublic void setVisibility(int visibility) {\r\n if (visibility == GONE || visibility == INVISIBLE) {\r\n clearAnimation();\r\n } else if (visibility == VISIBLE) {\r\n startAnimation(mRotation);\r\n }\r\n super.setVisibility(visibility);\r\n}\r\n</code></pre>\r\n\r\n<p>}</p>\r\n\r\n<p>rotate.xml</p>\r\n\r\n<pre><code> &lt;?xml version=""1.0"" encoding=""utf-8""?&gt;\r\n&lt;rotate xmlns:android=""http://schemas.android.com/apk/res/android""\r\n android:duration=""5000""\r\n android:fromDegrees=""0""\r\n android:interpolator=""@android:anim/linear_interpolator""\r\n android:pivotX=""50%""\r\n android:pivotY=""50%""\r\n android:startOffset=""0""\r\n android:toDegrees=""360"" /&gt;\r\n</code></pre>\r\n\r\n<p><strong>EDITED</strong></p>\r\n\r\n<pre><code> RotateImageView image = new RotateImageView(Context);\r\n image.addOnClickListener(new View.OnClickListener(){\r\n if(image.isAnimating)\r\n image.stopAnimating();\r\n else \r\n image.startAnimating();\r\n}\r\n</code></pre>"
340,0.081683,45123186,Find out an object's box actual 4 edges scaled to the screen/camera size (Unity 2D),45124972.0,"<p>As Draco18s pointed out, you convert <code>pos</code> to screenspace but not <code>rect</code>.</p>\r\n\r\n<p>If you look at unity's API docs for <code>WorldToScreenPoint()</code>\r\nit says that it </p>\r\n\r\n<blockquote>\r\n <p>Transforms position from world space into screen space. Screenspace is\r\n defined in pixels.</p>\r\n</blockquote>\r\n\r\n<p>whereas <code>transform.position</code> returns</p>\r\n\r\n<blockquote>\r\n <p>The position of the transform in world space.</p>\r\n</blockquote>\r\n\r\n<p>What you have is a problem of combining screenspace, measured in pixels, and world space, which is totally different.</p>\r\n\r\n<p>Because you are using <code>pixelRect</code>, I would recommend changing <code>var rect = image.transform.rect;</code>\r\nto be <code>var rect = Camera.main.WorldToScreenPoint(image.transform.position);</code></p>\r\n\r\n<p>You will then of course have to also check the other corners in a similar way, and if all 4 corners return true then the the object is inside the other.</p>"


In [318]:
similar_qs = find_similar(tfidf_matrix, "How do I create a bar chart?")
similar_qs

[(210, 0.3465426956980782),
 (267, 0.33679816934966145),
 (18, 0.29708062066447544)]

In [319]:
decorate_answers(similar_qs)

,cos_similarity,q_id,q_title,accepted_answer_id,a_body
210,0.346543,28157752,R: Stacked bar chart from two column data,28157933.0,"<p>You can use a base <code>barplot</code> here. Here an example</p>\r\n\r\n<pre><code>dd&lt;-read.table(text=""Date Speed\r\n01/01/2013 56\r\n01/01/2013 45\r\n01/01/2013 34\r\n02/01/2013 23\r\n02/01/2013 12\r\n02/01/2013 1\r\n03/01/2013 48"", header=T)\r\n\r\n#make sure it's a date \r\nspeed_date &lt;- as.Date(dd$Date, ""%m/%d/%Y"")\r\n\r\n#cut data into requested bins\r\nspeed_cut &lt;- cut(dd$Speed, \r\n breaks=c(0,40,48,60,Inf),\r\n labels=c(""0-40"", ""41-48"", ""49-60"", ""&gt; 60"")\r\n)\r\n\r\n#pick a color for each bin\r\nspeed_cols &lt;- heat.colors(nlevels(speed_cut))\r\n\r\nbarplot(table(speed_cut, speed_date), col=speed_cols)\r\nlegend(""topright"",levels(speed_cut), fill=speed_cols)\r\n</code></pre>\r\n\r\n<p>this produces</p>\r\n\r\n<p><img src=""https://i.stack.imgur.com/ioNuT.png"" alt=""enter image description here""></p>\r\n\r\n<p>You may wish to change the colors to suit your taste.</p>"
267,0.336798,46637560,to resize the existing lines in a bar chart,46637617.0,"<p>You need to update the existing paths:</p>\r\n\r\n<pre><code>g.selectAll("".line"")\r\n .transition()\r\n .duration(750)\r\n .attr(""d"", function(d) {\r\n return line(d.values)\r\n })\r\n</code></pre>\r\n\r\n<p>Here is the updated plunker: <a href=""http://plnkr.co/edit/w0INQyy3hEcuzvP5J8FR?p=preview"" rel=""nofollow noreferrer"">http://plnkr.co/edit/w0INQyy3hEcuzvP5J8FR?p=preview</a></p>"
18,0.297081,50667397,Plotting a DataFrame (that has a date index) as a bar chart,50667655.0,"<p>It seems to be working in the example below.</p>\r\n\r\n<pre><code>from numpy.random import randint\r\nimport pandas as pd\r\nimport matplotlib.pyplot as plt\r\n\r\n# Create data frames for the example\r\nrng = pd.date_range('3/7/2018 00:00', periods=10, freq='1w')\r\ndf = pd.DataFrame({'usersPerWeek': randint(10, 200, 10)}, index=rng)\r\n\r\nfigPerWeek = plt.figure(dpi=80, figsize=(8,1))\r\naxis = figPerWeek.add_subplot(1,1,1)\r\naxis.bar(x=df.index, height=df.usersPerWeek)\r\nplt.ylabel('usersPerWeek')\r\nplt.show()\r\n</code></pre>\r\n\r\n<p><a href=""https://i.stack.imgur.com/Xgypf.png"" rel=""nofollow noreferrer""><img src=""https://i.stack.imgur.com/Xgypf.png"" alt=""enter image description here""></a></p>"
